In [1]:
from IPython.display import display, HTML
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from scipy.stats import wilcoxon, mannwhitneyu, ks_2samp

from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from scipy.stats import pearsonr

# decision trees
from sklearn import tree
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.max_open_warning': 0})
import numpy as np
import io, os , sys, types
import copy

import seaborn as sns
sns.set(color_codes=True)

# importing custom notebooks here
import import_ipynb as imppy
sys.meta_path.append(imppy.NotebookFinder())
from analytics_functions import *
from starcon_functions import *
# import starcon_functions

# print in a tabulated pretty manner
from tabulate import tabulate

importing Jupyter notebook from analytics_functions.ipynb
importing Jupyter notebook from starcon_functions.ipynb


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
cwd = os.getcwd()
fname_raw = r'dataset\PlanetDatabase.csv'
data_raw = pd.read_csv(fname_raw)
data_raw = data_raw[data_raw.Cluster != 'Sol']  # Removing Sol system, since it is irrelevant
# data_raw = data_raw.iloc[:,1:].copy(deep=True)
fontsz = 14
DEBUG = False

In [4]:
cols_data = list(data_raw.columns.values)
col_class = 'MinValue' # expects [0, 1] values
cols_data.remove(col_class)
cols_numeric_all = list(data_raw.describe().columns.values) # all the numerical variables
cols_categoric_all = list(set(data_raw.columns.values) - set(cols_numeric_all) - set([col_class]))

cols_numeric = ['X', 'Y', 'DistSol', 'MinVolume', 'BioUnits', 'DistFromStar', ]
cols_categoric = ['Cluster', 'Star', 'StarColor', 'Fleet']
col_Planet = 'Planet'  # Need to convert this one to numeric

data_numeric = data_raw[cols_numeric].copy(deep=True)
data_categoric = data_raw[cols_categoric].copy(deep=True)
# Print dataset classes and distribution
# dataset_classes(data_raw, col_class)

In [13]:
def convert_RomanNumerals_Column_to_Int(df, colName):
    return df[colName].apply(int_from_roman_numeral)

def letters_to_numbers(ch):
    """Convert letter to letter number in the alphabet (a is 1, b is 2, and so on)"""
    if ch is None:
        return None
    
    ch = ch.lower()
    if ch.isalpha():
        return ord(ch)-97
    else:
        return None
    
def convert_moonDesignations_to_int(df, colName):
    return df[colName].apply(letters_to_numbers)

def convert_planet_to_planet_moon(df, colName, outputColsNamesList):
    """Converts a column with planet notation III-a, V-b, etc., to two columns [roman numeral, moon number] """
    """It then converts both the roman numerals and moon designation to integers"""
    """For example V-a would become [5, 1]"""
    if len(outputColsNamesList) is not 2:
        print("Warning: output column names must be a list of 2 names. Returning None")
        return None
    else:
        new_df = pd.DataFrame(df[colName].str.split('-',1).tolist(), columns = outputColsNamesList)
        new_df[outputColsNamesList[0]] = convert_RomanNumerals_Column_to_Int(new_df, outputColsNamesList[0])
        new_df[outputColsNamesList[1]] = convert_moonDesignations_to_int(new_df, outputColsNamesList[1])
        return new_df
    
   
    

In [14]:
convert_planet_to_planet_moon(data_raw, 'Planet', ['PlanetNumber', 'MoonNumber'])


,PlanetNumber,MoonNumber
0,1,NaN
1,2,NaN
2,3,NaN
3,3,0.0
4,3,1.0
5,3,2.0
6,3,3.0
7,4,NaN
8,4,0.0
9,4,1.0


In [7]:
int_from_roman_numeral('IV')

4